<a href="https://colab.research.google.com/github/paveethranswam/Speech-style-transfer-and-ASR/blob/main/Cycle_GAN_VC2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchaudio
import torchaudio.transforms
from torch.utils.data import DataLoader
import pdb
import torch.optim as optim
import tqdm


In [4]:
path = "/content/drive/My Drive/Projects and research stuffs/DLS Assignments/CycleGAN_VC_2/"


In [5]:
!pip install -r "/content/drive/My Drive/Projects and research stuffs/DLS Assignments/CycleGAN_VC_2/requirements.txt" 


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 214 kB 16.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pyworld: filename=pyworld-0.3.2-cp38-cp38-linux_x86_64.whl size=673642 sha256=04763caeb15cb77f6e1344662b9b71530a10f78844ace8ded1ac8b28cafbbc22
  Stored in directory: /root/.cache/pip/wheels/b7/b1/d2/8c78d691f7d5b0bb4ba9993926db209429c92686476837627f
Successfully built pyworld


In [ ]:
!python3 "/content/drive/My Drive/Projects and research stuffs/DLS Assignments/CycleGAN_VC_2/preprocess_training.py" --train_A_dir "/content/drive/My Drive/Projects and research stuffs/DLS Assignments/CycleGAN_VC_2/data/S0913/" --train_B_dir "/content/drive/My Drive/Projects and research stuffs/DLS Assignments/CycleGAN_VC_2/data/gaoxiaosong/" --cache_folder "/content/drive/My Drive/Projects and research stuffs/DLS Assignments/CycleGAN_VC_2/cache/"



Starting to prepocess data.......
Log Pitch A
Mean: 4.8262, Std: 0.1869
Log Pitch B
Mean: 4.6393, Std: 0.2312
Preprocessing finsihed!! see your directory ../cache for cached preprocessed data
Time taken for preprocessing 42.6606 seconds


In [ ]:
!python "/content/drive/My Drive/Projects and research stuffs/DLS Assignments/CycleGAN_VC_2/train.py" --logf0s_normalization "/content/drive/My Drive/Projects and research stuffs/DLS Assignments/CycleGAN_VC_2/cache/logf0s_normalization.npz" --mcep_normalization "/content/drive/My Drive/Projects and research stuffs/DLS Assignments/CycleGAN_VC_2/cache/mcep_normalization.npz" --coded_sps_A_norm "/content/drive/My Drive/Projects and research stuffs/DLS Assignments/CycleGAN_VC_2/cache/coded_sps_A_norm.pickle" --coded_sps_B_norm "/content/drive/My Drive/Projects and research stuffs/DLS Assignments/CycleGAN_VC_2/cache/coded_sps_B_norm.pickle" --model_checkpoint "/content/drive/My Drive/Projects and research stuffs/DLS Assignments/CycleGAN_VC_2/model_checkpoint/" --validation_A_dir "/content/drive/My Drive/Projects and research stuffs/DLS Assignments/CycleGAN_VC_2/data/S0913/" --output_A_dir "/content/drive/My Drive/Projects and research stuffs/DLS Assignments/CycleGAN_VC_2/converted_sound/S0913" --validation_B_dir "/content/drive/My Drive/Projects and research stuffs/DLS Assignments/CycleGAN_VC_2/data/gaoxiaosong/" --output_B_dir "/content/drive/My Drive/Projects and research stuffs/DLS Assignments/CycleGAN_VC_2/converted_sound/gaoxiaosong/"



Iter:7 Generator Loss:24.9478 Discrimator Loss:0.5103 GA2B:0.2566 GB2A:0.2571 G_id:1.6217 G_cyc:1.6325 D_A:0.2595 D_B:0.2502: : 0it [01:27, ?it/s]
Iter:7 Generator Loss:24.9478 Discrimator Loss:0.5103 GA2B:0.2566 GB2A:0.2571 G_id:1.6217 G_cyc:1.6325 D_A:0.2595 D_B:0.2502: : 0it [01:27, ?it/s]

Iter:9 Generator Loss:23.5700 Discrimator Loss:0.5073 GA2B:0.2570 GB2A:0.2577 G_id:1.5002 G_cyc:1.5554 D_A:0.2524 D_B:0.2546: : 0it [00:21, ?it/s]
Iter:11 Generator Loss:24.9432 Discrimator Loss:0.5019 GA2B:0.2654 GB2A:0.2602 G_id:1.5852 G_cyc:1.6492 D_A:0.2548 D_B:0.2462: : 0it [00:43, ?it/s]
Iter:13 Generator Loss:23.7613 Discrimator Loss:0.5004 GA2B:0.2695 GB2A:0.2610 G_id:1.5285 G_cyc:1.5588 D_A:0.2498 D_B:0.2495: : 0it [01:08, ?it/s]
Iter:15 Generator Loss:23.4053 Discrimator Loss:0.5037 GA2B:0.2713 GB2A:0.2688 G_id:1.4894 G_cyc:1.5418 D_A:0.2502 D_B:0.2520: : 0it [01:31, ?it/s]
Iter:23 Generator Loss:23.7256 Discrimator Loss:0.4464 GA2B:0.3150 GB2A:0.2988 G_id:1.5133 G_cyc:1.5545 D_A:0.2334

### Proper Pipeline: 
### 1. Proprocessing - audio to Mel spectrograms 
### 2. Initialize CycleGAN architecture 
### 3. Train and validate Model 
### 4. Convert Mel spectrograms back to Audio format.

In [5]:
# !unzip "./English Speakers Mini.zip" -d "./English Speakers Mini"

# !unzip "./Indian Speakers Mini.zip" -d "./Indian Speakers Mini"


In [6]:
import os
import time
import argparse
import numpy as np
import pickle
import librosa
import numpy as np
import os
import pyworld
from pprint import pprint
import librosa.display



#### Step 1: Preprocess- autio to Mel Spectrograms

In [7]:
# Use this to see how spectograms look:
def world_decompose(wav, fs, frame_period=5.0):
    # Decompose speech signal into f0, spectral envelope and aperiodicity using WORLD
    wav = wav.astype(np.float64)
    f0, timeaxis = pyworld.harvest(
        wav, fs, frame_period=frame_period, f0_floor=71.0, f0_ceil=800.0)

    # Finding Spectogram
    sp = pyworld.cheaptrick(wav, f0, timeaxis, fs)

    # Finding aperiodicity
    ap = pyworld.d4c(wav, f0, timeaxis, fs)

    # Use this in Ipython to see plot
    # librosa.display.specshow(np.log(sp).T,
    #                          sr=fs,
    #                          hop_length=int(0.001 * fs * frame_period),
    #                          x_axis="time",
    #                          y_axis="linear",
    #                          cmap="magma")
    # colorbar()
    return f0, timeaxis, sp, ap

In [8]:
def load_wavs(wav_dir, sr):
    wavs = list()
    for file in os.listdir(wav_dir):
        file_path = os.path.join(wav_dir, file)
        wav, _ = librosa.load(file_path, sr=sr, mono=True)
        # wav = wav.astype(np.float64)
        wavs.append(wav)
    return wavs


def world_decompose(wav, fs, frame_period=5.0):
    # Decompose speech signal into f0, spectral envelope and aperiodicity using WORLD
    wav = wav.astype(np.float64)
    f0, timeaxis = pyworld.harvest(
        wav, fs, frame_period=frame_period, f0_floor=71.0, f0_ceil=800.0)

    # Finding Spectogram
    sp = pyworld.cheaptrick(wav, f0, timeaxis, fs)

    # Finding aperiodicity
    ap = pyworld.d4c(wav, f0, timeaxis, fs)

    # Use this in Ipython to see plot
    # librosa.display.specshow(np.log(sp).T,
    #                          sr=fs,
    #                          hop_length=int(0.001 * fs * frame_period),
    #                          x_axis="time",
    #                          y_axis="linear",
    #                          cmap="magma")
    # colorbar()
    return f0, timeaxis, sp, ap


def world_encode_spectral_envelop(sp, fs, dim=24):
    # Get Mel-Cepstral coefficients (MCEPs)
    # sp = sp.astype(np.float64)
    coded_sp = pyworld.code_spectral_envelope(sp, fs, dim)
    return coded_sp


def world_encode_data(wave, fs, frame_period=5.0, coded_dim=24):
    f0s = list()
    timeaxes = list()
    sps = list()
    aps = list()
    coded_sps = list()
    for wav in wave:
        f0, timeaxis, sp, ap = world_decompose(wav=wav,
                                              fs=fs,
                                              frame_period=frame_period)
        coded_sp = world_encode_spectral_envelop(sp=sp, fs=fs, dim=coded_dim)
        f0s.append(f0)
        timeaxes.append(timeaxis)
        sps.append(sp)
        aps.append(ap)
        coded_sps.append(coded_sp)
    return f0s, timeaxes, sps, aps, coded_sps


def logf0_statistics(f0s):
    # Note: np.ma.log() calculating log on masked array (for incomplete or invalid entries in array)
    log_f0s_concatenated = np.ma.log(np.concatenate(f0s))
    log_f0s_mean = log_f0s_concatenated.mean()
    log_f0s_std = log_f0s_concatenated.std()
    return log_f0s_mean, log_f0s_std


def transpose_in_list(lst):
    transposed_lst = list()
    for array in lst:
        transposed_lst.append(array.T)
    return transposed_lst


def coded_sps_normalization_fit_transform(coded_sps):
    coded_sps_concatenated = np.concatenate(coded_sps, axis=1)
    coded_sps_mean = np.mean(coded_sps_concatenated, axis=1, keepdims=True)
    coded_sps_std = np.std(coded_sps_concatenated, axis=1, keepdims=True)
    coded_sps_normalized = list()
    for coded_sp in coded_sps:
        coded_sps_normalized.append(
            (coded_sp - coded_sps_mean) / coded_sps_std)
    return coded_sps_normalized, coded_sps_mean, coded_sps_std


def wav_padding(wav, sr, frame_period, multiple=4):
    assert wav.ndim == 1
    num_frames = len(wav)
    num_frames_padded = int((np.ceil((np.floor(num_frames / (sr * frame_period / 1000)) +
                                      1) / multiple + 1) * multiple - 1) * (sr * frame_period / 1000))
    num_frames_diff = num_frames_padded - num_frames
    num_pad_left = num_frames_diff // 2
    num_pad_right = num_frames_diff - num_pad_left
    wav_padded = np.pad(wav, (num_pad_left, num_pad_right),
                        'constant', constant_values=0)

    return wav_padded


def pitch_conversion(f0, mean_log_src, std_log_src, mean_log_target, std_log_target):
    # Logarithm Gaussian Normalization for Pitch Conversions
    f0_converted = np.exp((np.log(f0) - mean_log_src) /
                          std_log_src * std_log_target + mean_log_target)
    return f0_converted


def world_decode_spectral_envelop(coded_sp, fs):
    fftlen = pyworld.get_cheaptrick_fft_size(fs)
    decoded_sp = pyworld.decode_spectral_envelope(coded_sp, fs, fftlen)
    return decoded_sp


def world_speech_synthesis(f0, decoded_sp, ap, fs, frame_period):
    wav = pyworld.synthesize(f0, decoded_sp, ap, fs, frame_period)
    wav = wav.astype(np.float32)
    return wav


def sample_train_data(dataset_A, dataset_B, n_frames=128):
    # Created Pytorch custom dataset instead
    num_samples = min(len(dataset_A), len(dataset_B))
    train_data_A_idx = np.arange(len(dataset_A))
    train_data_B_idx = np.arange(len(dataset_B))
    np.random.shuffle(train_data_A_idx)
    np.random.shuffle(train_data_B_idx)
    train_data_A_idx_subset = train_data_A_idx[:num_samples]
    train_data_B_idx_subset = train_data_B_idx[:num_samples]

    train_data_A = list()
    train_data_B = list()

    for idx_A, idx_B in zip(train_data_A_idx_subset, train_data_B_idx_subset):
        data_A = dataset_A[idx_A]
        frames_A_total = data_A.shape[1]
        assert frames_A_total >= n_frames
        start_A = np.random.randint(frames_A_total - n_frames + 1)
        end_A = start_A + n_frames
        train_data_A.append(data_A[:, start_A:end_A])

        data_B = dataset_B[idx_B]
        frames_B_total = data_B.shape[1]
        assert frames_B_total >= n_frames
        start_B = np.random.randint(frames_B_total - n_frames + 1)
        end_B = start_B + n_frames
        train_data_B.append(data_B[:, start_B:end_B])

    train_data_A = np.array(train_data_A)
    train_data_B = np.array(train_data_B)

    return train_data_A, train_data_B







In [9]:
import sys
sys.path.append("/content/drive/My Drive/Projects and research stuffs/DLS Assignments/CycleGAN_VC_2/")



In [10]:
# Custom Classes

import preprocess # I changed this
# preprocess = Preprocess()

def save_pickle(variable, fileName):
    with open(fileName, 'wb') as f:
        pickle.dump(variable, f)


def load_pickle_file(fileName):
    with open(fileName, 'rb') as f:
        return pickle.load(f)


def preprocess_for_training(train_A_dir, train_B_dir, cache_folder):
    num_mcep = 36
    sampling_rate = 16000
    frame_period = 5.0
    n_frames = 128

    print("Starting to prepocess data.......")
    start_time = time.time()
    wavs_A = preprocess.load_wavs(wav_dir=train_A_dir, sr=sampling_rate)
    wavs_B = preprocess.load_wavs(wav_dir=train_B_dir, sr=sampling_rate)

    f0s_A, timeaxes_A, sps_A, aps_A, coded_sps_A = preprocess.world_encode_data(
        wave=wavs_A, fs=sampling_rate, frame_period=frame_period, coded_dim=num_mcep)
    f0s_B, timeaxes_B, sps_B, aps_B, coded_sps_B = preprocess.world_encode_data(
        wave=wavs_B, fs=sampling_rate, frame_period=frame_period, coded_dim=num_mcep)

    log_f0s_mean_A, log_f0s_std_A = preprocess.logf0_statistics(f0s=f0s_A)
    log_f0s_mean_B, log_f0s_std_B = preprocess.logf0_statistics(f0s=f0s_B)

    print("Log Pitch A")
    print("Mean: {:.4f}, Std: {:.4f}".format(log_f0s_mean_A, log_f0s_std_A))
    print("Log Pitch B")
    print("Mean: {:.4f}, Std: {:.4f}".format(log_f0s_mean_B, log_f0s_std_B))

    coded_sps_A_transposed = preprocess.transpose_in_list(lst=coded_sps_A)
    coded_sps_B_transposed = preprocess.transpose_in_list(lst=coded_sps_B)

    coded_sps_A_norm, coded_sps_A_mean, coded_sps_A_std = preprocess.coded_sps_normalization_fit_transform(
        coded_sps=coded_sps_A_transposed)
    coded_sps_B_norm, coded_sps_B_mean, coded_sps_B_std = preprocess.coded_sps_normalization_fit_transform(
        coded_sps=coded_sps_B_transposed)

    if not os.path.exists(cache_folder):
        os.makedirs(cache_folder)

    np.savez(os.path.join(cache_folder, 'logf0s_normalization.npz'),
             mean_A=log_f0s_mean_A,
             std_A=log_f0s_std_A,
             mean_B=log_f0s_mean_B,
             std_B=log_f0s_std_B)

    np.savez(os.path.join(cache_folder, 'mcep_normalization.npz'),
             mean_A=coded_sps_A_mean,
             std_A=coded_sps_A_std,
             mean_B=coded_sps_B_mean,
             std_B=coded_sps_B_std)

    save_pickle(variable=coded_sps_A_norm,
                fileName=os.path.join(cache_folder, "coded_sps_A_norm.pickle"))
    save_pickle(variable=coded_sps_B_norm,
                fileName=os.path.join(cache_folder, "coded_sps_B_norm.pickle"))

    end_time = time.time()
    print("Preprocessing finsihed!! see your directory ../cache for cached preprocessed data")

    print("Time taken for preprocessing {:.4f} seconds".format(
        end_time - start_time))


In [11]:
# I changed this

# if __name__ == '__main__':
print('Prepare data for training Cycle GAN using PyTorch')

# train_A_dir_default = "/content/drive/My Drive/Projects and research stuffs/DLS Assignments/CycleGAN_VC_2/data/S0913/"
# train_B_dir_default = "/content/drive/My Drive/Projects and research stuffs/DLS Assignments/CycleGAN_VC_2/data/gaoxiaosong/"

train_A_dir_default = "/content/drive/My Drive/Projects and research stuffs/DLS Assignments/CycleGAN_VC_2/data/Ind/"
train_B_dir_default = "/content/drive/My Drive/Projects and research stuffs/DLS Assignments/CycleGAN_VC_2/data/Eng/"

cache_folder_default = "/content/drive/My Drive/Projects and research stuffs/DLS Assignments/CycleGAN_VC_2/cache/"

# parser.add_argument('--train_A_dir', type=str,
#                     help="Directory for source voice sample", default=train_A_dir_default)
# parser.add_argument('--train_B_dir', type=str,
#                     help="Directory for target voice sample", default=train_B_dir_default)
# parser.add_argument('--cache_folder', type=str,
#                     help="Store preprocessed data in cache folders", default=cache_folder_default)
# argv = parser.parse_args()

# train_A_dir = argv.train_A_dir
# train_B_dir = argv.train_B_dir
# cache_folder = argv.cache_folder

preprocess_for_training(train_A_dir_default, train_B_dir_default, cache_folder_default)


Prepare data for training Cycle GAN using PyTorch
Starting to prepocess data.......
Log Pitch A
Mean: 5.4307, Std: 0.2897
Log Pitch B
Mean: 5.1966, Std: 0.2484
Preprocessing finsihed!! see your directory ../cache for cached preprocessed data
Time taken for preprocessing 506.5709 seconds


#### Step 2: Model architecture define and run it

In [12]:
# Get device
if(torch.cuda.is_available()):
    device = torch.device('cuda')
    print(f"Using {device} device")
    print(torch.cuda.get_device_name(0))

# Kaiming He initialization
def he_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.kaiming_uniform_(m.weight, a=0, mode='fan_in', nonlinearity='relu')
        m.bias.data.fill_(0)
        
     # Now add for conv layer also
    elif isinstance(m, nn.Conv2d):
        torch.nn.init.kaiming_uniform_(m.weight, a=0, mode='fan_in', nonlinearity='relu')
        m.bias.data.fill_(0)
        

Using cuda device
Tesla T4


In [20]:
import os
import numpy as np
import argparse
import torch
import time
import librosa
import pickle
import soundfile as sf

import preprocess
from trainingDataset import trainingDataset
from model_tf import Generator, Discriminator
from tqdm import tqdm

os.environ["CUDA_VISIBLE_DEVICES"] = "3"


class CycleGANTraining(object):
    def __init__(self,
                 logf0s_normalization,
                 mcep_normalization,
                 coded_sps_A_norm,
                 coded_sps_B_norm,
                 model_checkpoint,
                 validation_A_dir,
                 output_A_dir,
                 validation_B_dir,
                 output_B_dir,
                 restart_training_at=None):
        self.start_epoch = 0
        self.num_epochs = 200  # 5000
        self.mini_batch_size = 1  # 1
        self.dataset_A = self.loadPickleFile(coded_sps_A_norm)
        self.dataset_B = self.loadPickleFile(coded_sps_B_norm)
        self.device = torch.device(
            'cuda' if torch.cuda.is_available() else 'cpu')
        print('Using device: ', self.device)

        # Speech Parameters
        logf0s_normalization = np.load(logf0s_normalization)
        self.log_f0s_mean_A = logf0s_normalization['mean_A']
        self.log_f0s_std_A = logf0s_normalization['std_A']
        self.log_f0s_mean_B = logf0s_normalization['mean_B']
        self.log_f0s_std_B = logf0s_normalization['std_B']

        mcep_normalization = np.load(mcep_normalization)
        self.coded_sps_A_mean = mcep_normalization['mean_A']
        self.coded_sps_A_std = mcep_normalization['std_A']
        self.coded_sps_B_mean = mcep_normalization['mean_B']
        self.coded_sps_B_std = mcep_normalization['std_B']

        # Generator and Discriminator
        self.generator_A2B = Generator().to(self.device)
        self.generator_B2A = Generator().to(self.device)
        self.discriminator_A = Discriminator().to(self.device)
        self.discriminator_B = Discriminator().to(self.device)

        # Loss Functions
        criterion_mse = torch.nn.MSELoss()

        # Optimizer
        g_params = list(self.generator_A2B.parameters()) + \
                   list(self.generator_B2A.parameters())
        d_params = list(self.discriminator_A.parameters()) + \
                   list(self.discriminator_B.parameters())

        # Initial learning rates
        self.generator_lr = 2e-4  # 0.0002
        self.discriminator_lr = 1e-4  # 0.0001

        # Learning rate decay
        self.generator_lr_decay = self.generator_lr / 200000
        self.discriminator_lr_decay = self.discriminator_lr / 200000

        # Starts learning rate decay from after this many iterations have passed
        self.start_decay = 10000  # 200000

        self.generator_optimizer = torch.optim.Adam(
            g_params, lr=self.generator_lr, betas=(0.5, 0.999))
        self.discriminator_optimizer = torch.optim.Adam(
            d_params, lr=self.discriminator_lr, betas=(0.5, 0.999))

        # To Load save previously saved models
        self.modelCheckpoint = model_checkpoint
        os.makedirs(self.modelCheckpoint, exist_ok=True)

        # Validation set Parameters
        self.validation_A_dir = validation_A_dir
        self.output_A_dir = output_A_dir
        os.makedirs(self.output_A_dir, exist_ok=True)
        self.validation_B_dir = validation_B_dir
        self.output_B_dir = output_B_dir
        os.makedirs(self.output_B_dir, exist_ok=True)

        # Storing Discriminatior and Generator Loss
        self.generator_loss_store = []
        self.discriminator_loss_store = []

        self.file_name = 'log_store_non_sigmoid.txt'

        if restart_training_at is not None:
            # Training will resume from previous checkpoint
            self.start_epoch = self.loadModel(restart_training_at)
            print("Training resumed")

    def adjust_lr_rate(self, optimizer, name='generator'):
        if name == 'generator':
            self.generator_lr = max(
                0., self.generator_lr - self.generator_lr_decay)
            for param_groups in optimizer.param_groups:
                param_groups['lr'] = self.generator_lr
        else:
            self.discriminator_lr = max(
                0., self.discriminator_lr - self.discriminator_lr_decay)
            for param_groups in optimizer.param_groups:
                param_groups['lr'] = self.discriminator_lr

    def reset_grad(self):
        self.generator_optimizer.zero_grad()
        self.discriminator_optimizer.zero_grad()

    def train(self):
        # Training Begins
        for epoch in range(self.start_epoch, self.num_epochs):
            start_time_epoch = time.time()

            # Constants
            cycle_loss_lambda = 10
            identity_loss_lambda = 5

            # Preparing Dataset
            n_samples = len(self.dataset_A)

            dataset = trainingDataset(datasetA=self.dataset_A,
                                      datasetB=self.dataset_B,
                                      n_frames=128)
            train_loader = torch.utils.data.DataLoader(dataset=dataset,
                                                       batch_size=self.mini_batch_size,
                                                       shuffle=True,
                                                       drop_last=False)

            pbar = tqdm(enumerate(train_loader))
            for i, (real_A, real_B) in enumerate(train_loader):
                num_iterations = (n_samples // self.mini_batch_size) * epoch + i
                # print("iteration no: ", num_iterations, epoch)

                if num_iterations > 10000:
                    identity_loss_lambda = 0
                if num_iterations > self.start_decay:
                    self.adjust_lr_rate(
                        self.generator_optimizer, name='generator')
                    self.adjust_lr_rate(
                        self.generator_optimizer, name='discriminator')

                real_A = real_A.to(self.device).float()
                real_B = real_B.to(self.device).float()

                # Generator Loss function

                fake_B = self.generator_A2B(real_A)
                cycle_A = self.generator_B2A(fake_B)

                fake_A = self.generator_B2A(real_B)
                cycle_B = self.generator_A2B(fake_A)

                identity_A = self.generator_B2A(real_A)
                identity_B = self.generator_A2B(real_B)

                d_fake_A = self.discriminator_A(fake_A)
                d_fake_B = self.discriminator_B(fake_B)

                # for the second step adverserial loss
                d_fake_cycle_A = self.discriminator_A(cycle_A)
                d_fake_cycle_B = self.discriminator_B(cycle_B)

                # Generator Cycle loss
                cycleLoss = torch.mean(
                    torch.abs(real_A - cycle_A)) + torch.mean(torch.abs(real_B - cycle_B))

                # Generator Identity Loss
                identiyLoss = torch.mean(
                    torch.abs(real_A - identity_A)) + torch.mean(torch.abs(real_B - identity_B))

                # Generator Loss
                generator_loss_A2B = torch.mean((1 - d_fake_B) ** 2)
                generator_loss_B2A = torch.mean((1 - d_fake_A) ** 2)

                # Total Generator Loss
                generator_loss = generator_loss_A2B + generator_loss_B2A + \
                                 cycle_loss_lambda * cycleLoss + identity_loss_lambda * identiyLoss
                self.generator_loss_store.append(generator_loss.item())

                # Backprop for Generator
                self.reset_grad()
                generator_loss.backward()

                # if num_iterations > self.start_decay:  # Linearly decay learning rate
                #     self.adjust_lr_rate(
                #         self.generator_optimizer, name='generator')

                self.generator_optimizer.step()

                # Discriminator Loss Function

                # Discriminator Feed Forward
                d_real_A = self.discriminator_A(real_A)
                d_real_B = self.discriminator_B(real_B)

                generated_A = self.generator_B2A(real_B)
                d_fake_A = self.discriminator_A(generated_A)

                # for the second step adverserial loss
                cycled_B = self.generator_A2B(generated_A)
                d_cycled_B = self.discriminator_B(cycled_B)

                generated_B = self.generator_A2B(real_A)
                d_fake_B = self.discriminator_B(generated_B)

                # for the second step adverserial loss
                cycled_A = self.generator_B2A(generated_B)
                d_cycled_A = self.discriminator_A(cycled_A)

                # Loss Functions
                d_loss_A_real = torch.mean((1 - d_real_A) ** 2)
                d_loss_A_fake = torch.mean((0 - d_fake_A) ** 2)
                d_loss_A = (d_loss_A_real + d_loss_A_fake) / 2.0

                d_loss_B_real = torch.mean((1 - d_real_B) ** 2)
                d_loss_B_fake = torch.mean((0 - d_fake_B) ** 2)
                d_loss_B = (d_loss_B_real + d_loss_B_fake) / 2.0

                # the second step adverserial loss
                d_loss_A_cycled = torch.mean((0 - d_cycled_A) ** 2)
                d_loss_B_cycled = torch.mean((0 - d_cycled_B) ** 2)
                d_loss_A_2nd = (d_loss_A_real + d_loss_A_cycled) / 2.0
                d_loss_B_2nd = (d_loss_B_real + d_loss_B_cycled) / 2.0

                # Final Loss for discriminator with the second step adverserial loss
                d_loss = (d_loss_A + d_loss_B) / 2.0 + (d_loss_A_2nd + d_loss_B_2nd) / 2.0
                self.discriminator_loss_store.append(d_loss.item())

                # Backprop for Discriminator
                self.reset_grad()
                d_loss.backward()

                # if num_iterations > self.start_decay:  # Linearly decay learning rate
                #     self.adjust_lr_rate(
                #         self.discriminator_optimizer, name='discriminator')

                self.discriminator_optimizer.step()
                print('\n\n')
                print("Epoch: {} Generator Loss: {:.4f} Discriminator Loss: {}\n\n".format(
                    epoch, generator_loss.item(), d_loss.item()) )
                
                if (i + 1) % 2 == 0:
                    pbar.set_description(
                        "Iter:{} Generator Loss:{:.4f} Discrimator Loss:{:.4f} GA2B:{:.4f} GB2A:{:.4f} G_id:{:.4f} G_cyc:{:.4f} D_A:{:.4f} D_B:{:.4f}".format(
                            num_iterations,
                            generator_loss.item(),
                            # loss['generator_loss'],
                            d_loss.item(), generator_loss_A2B, generator_loss_B2A, identiyLoss, cycleLoss, d_loss_A,
                            d_loss_B))

            #                 if num_iterations % 50 == 0:
            #                     store_to_file = "Iter:{}\t Generator Loss:{:.4f} Discrimator Loss:{:.4f} \tGA2B:{:.4f} GB2A:{:.4f} G_id:{:.4f} G_cyc:{:.4f} D_A:{:.4f} D_B:{:.4f}".format(
            #                         num_iterations, generator_loss.item(), d_loss.item(), generator_loss_A2B, generator_loss_B2A,
            #                         identiyLoss, cycleLoss, d_loss_A, d_loss_B)
            #                     print(
            #                         "Iter:{}\t Generator Loss:{:.4f} Discrimator Loss:{:.4f} \tGA2B:{:.4f} GB2A:{:.4f} G_id:{:.4f} G_cyc:{:.4f} D_A:{:.4f} D_B:{:.4f}".format(
            #                             num_iterations, generator_loss.item(), d_loss.item(), generator_loss_A2B,
            #                             generator_loss_B2A, identiyLoss, cycleLoss, d_loss_A, d_loss_B))
            #                     self.store_to_file(store_to_file)

            #             end_time = time.time()
            #             store_to_file = "Epoch: {} Generator Loss: {:.4f} Discriminator Loss: {}, Time: {:.2f}\n\n".format(
            #                 epoch, generator_loss.item(), d_loss.item(), end_time - start_time_epoch)
            #             self.store_to_file(store_to_file)
            #             print("Epoch: {} Generator Loss: {:.4f} Discriminator Loss: {}, Time: {:.2f}\n\n".format(
            #                 epoch, generator_loss.item(), d_loss.item(), end_time - start_time_epoch))

            
            
            if epoch % 50 == 0 and epoch != 0:
                end_time = time.time()
                store_to_file = "Epoch: {} Generator Loss: {:.4f} Discriminator Loss: {}, Time: {:.2f}\n\n".format(
                    epoch, generator_loss.item(), d_loss.item(), end_time - start_time_epoch)
                self.store_to_file(store_to_file)
                print("Epoch: {} Generator Loss: {:.4f} Discriminator Loss: {}, Time: {:.2f}\n\n".format(
                    epoch, generator_loss.item(), d_loss.item(), end_time - start_time_epoch))

                # Save the Entire model
                print("Saving model Checkpoint  ......")
                store_to_file = "Saving model Checkpoint  ......"
                self.store_to_file(store_to_file)
                self.saveModelCheckPoint(epoch, '{}'.format(
                    self.modelCheckpoint + '_CycleGAN_CheckPoint'))
                print("Model Saved!")

            if epoch % 50 == 0 and epoch != 0:
                # Validation Set
                validation_start_time = time.time()
                self.validation_for_A_dir()
                self.validation_for_B_dir()
                validation_end_time = time.time()
                store_to_file = "Time taken for validation Set: {}".format(
                    validation_end_time - validation_start_time)
                self.store_to_file(store_to_file)
                print("Time taken for validation Set: {}".format(
                    validation_end_time - validation_start_time))

    def validation_for_A_dir(self):
        num_mcep = 36
        sampling_rate = 16000
        frame_period = 5.0
        n_frames = 128
        validation_A_dir = self.validation_A_dir
        output_A_dir = self.output_A_dir

        print("Generating Validation Data B from A...")
        for file in os.listdir(validation_A_dir):
            filePath = os.path.join(validation_A_dir, file)
            wav, _ = librosa.load(filePath, sr=sampling_rate, mono=True)
            wav = preprocess.wav_padding(wav=wav,
                                         sr=sampling_rate,
                                         frame_period=frame_period,
                                         multiple=4)
            f0, timeaxis, sp, ap = preprocess.world_decompose(
                wav=wav, fs=sampling_rate, frame_period=frame_period)
            f0_converted = preprocess.pitch_conversion(f0=f0,
                                                       mean_log_src=self.log_f0s_mean_A,
                                                       std_log_src=self.log_f0s_std_A,
                                                       mean_log_target=self.log_f0s_mean_B,
                                                       std_log_target=self.log_f0s_std_B)
            coded_sp = preprocess.world_encode_spectral_envelop(
                sp=sp, fs=sampling_rate, dim=num_mcep)
            coded_sp_transposed = coded_sp.T
            coded_sp_norm = (coded_sp_transposed -
                             self.coded_sps_A_mean) / self.coded_sps_A_std
            coded_sp_norm = np.array([coded_sp_norm])

            if torch.cuda.is_available():
                coded_sp_norm = torch.from_numpy(coded_sp_norm).cuda().float()
            else:
                coded_sp_norm = torch.from_numpy(coded_sp_norm).float()

            coded_sp_converted_norm = self.generator_A2B(coded_sp_norm)
            coded_sp_converted_norm = coded_sp_converted_norm.cpu().detach().numpy()
            coded_sp_converted_norm = np.squeeze(coded_sp_converted_norm)
            coded_sp_converted = coded_sp_converted_norm * \
                                 self.coded_sps_B_std + self.coded_sps_B_mean
            coded_sp_converted = coded_sp_converted.T
            coded_sp_converted = np.ascontiguousarray(coded_sp_converted)
            decoded_sp_converted = preprocess.world_decode_spectral_envelop(
                coded_sp=coded_sp_converted, fs=sampling_rate)
            wav_transformed = preprocess.world_speech_synthesis(f0=f0_converted,
                                                                decoded_sp=decoded_sp_converted,
                                                                ap=ap,
                                                                fs=sampling_rate,
                                                                frame_period=frame_period)
            # librosa.output.write_wav(path=os.path.join(output_A_dir, os.path.basename(file)),
            #                          y=wav_transformed,
            #                          sr=sampling_rate)
            sf.write(os.path.join(output_A_dir, os.path.basename(file)), wav_transformed, sampling_rate) # a wav at sampling rate 1 Hz

    def validation_for_B_dir(self):
        num_mcep = 36
        sampling_rate = 16000
        frame_period = 5.0
        n_frames = 128
        validation_B_dir = self.validation_B_dir
        output_B_dir = self.output_B_dir

        print("Generating Validation Data A from B...")
        for file in os.listdir(validation_B_dir):
            filePath = os.path.join(validation_B_dir, file)
            wav, _ = librosa.load(filePath, sr=sampling_rate, mono=True)
            wav = preprocess.wav_padding(wav=wav,
                                         sr=sampling_rate,
                                         frame_period=frame_period,
                                         multiple=4)
            f0, timeaxis, sp, ap = preprocess.world_decompose(
                wav=wav, fs=sampling_rate, frame_period=frame_period)
            f0_converted = preprocess.pitch_conversion(f0=f0,
                                                       mean_log_src=self.log_f0s_mean_B,
                                                       std_log_src=self.log_f0s_std_B,
                                                       mean_log_target=self.log_f0s_mean_A,
                                                       std_log_target=self.log_f0s_std_A)
            coded_sp = preprocess.world_encode_spectral_envelop(
                sp=sp, fs=sampling_rate, dim=num_mcep)
            coded_sp_transposed = coded_sp.T
            coded_sp_norm = (coded_sp_transposed -
                             self.coded_sps_B_mean) / self.coded_sps_B_std
            coded_sp_norm = np.array([coded_sp_norm])

            if torch.cuda.is_available():
                coded_sp_norm = torch.from_numpy(coded_sp_norm).cuda().float()
            else:
                coded_sp_norm = torch.from_numpy(coded_sp_norm).float()

            coded_sp_converted_norm = self.generator_B2A(coded_sp_norm)
            coded_sp_converted_norm = coded_sp_converted_norm.cpu().detach().numpy()
            coded_sp_converted_norm = np.squeeze(coded_sp_converted_norm)
            coded_sp_converted = coded_sp_converted_norm * \
                                 self.coded_sps_A_std + self.coded_sps_A_mean
            coded_sp_converted = coded_sp_converted.T
            coded_sp_converted = np.ascontiguousarray(coded_sp_converted)
            decoded_sp_converted = preprocess.world_decode_spectral_envelop(
                coded_sp=coded_sp_converted, fs=sampling_rate)
            wav_transformed = preprocess.world_speech_synthesis(f0=f0_converted,
                                                                decoded_sp=decoded_sp_converted,
                                                                ap=ap,
                                                                fs=sampling_rate,
                                                                frame_period=frame_period)
            # librosa.output.write_wav(path=os.path.join(output_B_dir, os.path.basename(file)),
            #                          y=wav_transformed,
            #                          sr=sampling_rate)
            
            sf.write(os.path.join(output_B_dir, os.path.basename(file)), wav_transformed, sampling_rate) # a wav at sampling rate 1 Hz

    def savePickle(self, variable, fileName):
        with open(fileName, 'wb') as f:
            pickle.dump(variable, f)

    def loadPickleFile(self, fileName):
        with open(fileName, 'rb') as f:
            return pickle.load(f)

    def store_to_file(self, doc):
        doc = doc + "\n"
        with open(self.file_name, "a") as myfile:
            myfile.write(doc)

    def saveModelCheckPoint(self, epoch, PATH):
        torch.save({
            'epoch': epoch,
            'generator_loss_store': self.generator_loss_store,
            'discriminator_loss_store': self.discriminator_loss_store,
            'model_genA2B_state_dict': self.generator_A2B.state_dict(),
            'model_genB2A_state_dict': self.generator_B2A.state_dict(),
            'model_discriminatorA': self.discriminator_A.state_dict(),
            'model_discriminatorB': self.discriminator_B.state_dict(),
            'generator_optimizer': self.generator_optimizer.state_dict(),
            'discriminator_optimizer': self.discriminator_optimizer.state_dict()
        }, PATH)

    def loadModel(self, PATH):
        checkPoint = torch.load(PATH)
        self.generator_A2B.load_state_dict(
            state_dict=checkPoint['model_genA2B_state_dict'])
        self.generator_B2A.load_state_dict(
            state_dict=checkPoint['model_genB2A_state_dict'])
        self.discriminator_A.load_state_dict(
            state_dict=checkPoint['model_discriminatorA'])
        self.discriminator_B.load_state_dict(
            state_dict=checkPoint['model_discriminatorB'])
        self.generator_optimizer.load_state_dict(
            state_dict=checkPoint['generator_optimizer'])
        self.discriminator_optimizer.load_state_dict(
            state_dict=checkPoint['discriminator_optimizer'])
        epoch = int(checkPoint['epoch']) + 1
        self.generator_loss_store = checkPoint['generator_loss_store']
        self.discriminator_loss_store = checkPoint['discriminator_loss_store']
        return epoch


# if __name__ == '__main__':
    # parser = argparse.ArgumentParser(
    #     description="Train CycleGAN using source dataset and target dataset")


In [ ]:
logf0s_normalization_default = "/content/drive/My Drive/Projects and research stuffs/DLS Assignments/CycleGAN_VC_2/cache/logf0s_normalization.npz"
mcep_normalization_default = "/content/drive/My Drive/Projects and research stuffs/DLS Assignments/CycleGAN_VC_2/cache/mcep_normalization.npz"
coded_sps_A_norm = "/content/drive/My Drive/Projects and research stuffs/DLS Assignments/CycleGAN_VC_2/cache/coded_sps_A_norm.pickle"
coded_sps_B_norm = "/content/drive/My Drive/Projects and research stuffs/DLS Assignments/CycleGAN_VC_2/cache/coded_sps_B_norm.pickle"
model_checkpoint = "/content/drive/My Drive/Projects and research stuffs/DLS Assignments/CycleGAN_VC_2/model_checkpoint/"
resume_training_at = "/content/drive/My Drive/Projects and research stuffs/DLS Assignments/CycleGAN_VC_2/model_checkpoint/_CycleGAN_CheckPoint"
resume_training_at = None

# validation_A_dir_default = "/content/drive/My Drive/Projects and research stuffs/DLS Assignments/CycleGAN_VC_2/data/S0913/"
validation_A_dir_default = "/content/drive/My Drive/Projects and research stuffs/DLS Assignments/CycleGAN_VC_2/data/Ind/"
output_A_dir_default = "/content/drive/My Drive/Projects and research stuffs/DLS Assignments/CycleGAN_VC_2/converted_sound/Ind"

# validation_B_dir_default = "/content/drive/My Drive/Projects and research stuffs/DLS Assignments/CycleGAN_VC_2/data/gaoxiaosong/"
validation_B_dir_default = "/content/drive/My Drive/Projects and research stuffs/DLS Assignments/CycleGAN_VC_2/data/Eng/"
output_B_dir_default = "/content/drive/My Drive/Projects and research stuffs/DLS Assignments/CycleGAN_VC_2/converted_sound/Eng/"

logf0s_normalization = logf0s_normalization_default
mcep_normalization = mcep_normalization_default
coded_sps_A_norm = coded_sps_A_norm
coded_sps_B_norm = coded_sps_B_norm
model_checkpoint = model_checkpoint
resume_training_at = resume_training_at

validation_A_dir = validation_A_dir_default
output_A_dir = output_A_dir_default
validation_B_dir = validation_B_dir_default
output_B_dir = output_B_dir_default

# Check whether following cached files exists
if not os.path.exists(logf0s_normalization) or not os.path.exists(mcep_normalization):
    print(
        "Cached files do not exist, please run the program preprocess_training.py first")

cycleGAN = CycleGANTraining(logf0s_normalization=logf0s_normalization,
                            mcep_normalization=mcep_normalization,
                            coded_sps_A_norm=coded_sps_A_norm,
                            coded_sps_B_norm=coded_sps_B_norm,
                            model_checkpoint=model_checkpoint,
                            validation_A_dir=validation_A_dir,
                            output_A_dir=output_A_dir,
                            validation_B_dir=validation_B_dir,
                            output_B_dir=output_B_dir,
                            restart_training_at=resume_training_at)
cycleGAN.train()

Using device:  cuda




0it [00:00, ?it/s]




Epoch: 0 Generator Loss: 26.8060 Discriminator Loss: 0.517740786075592






Iter:1 Generator Loss:33.4445 Discrimator Loss:0.5191 GA2B:0.2736 GB2A:0.3239 G_id:2.1929 G_cyc:2.1882 D_A:0.2614 D_B:0.2577: : 0it [00:00, ?it/s]




Epoch: 0 Generator Loss: 33.4445 Discriminator Loss: 0.519097626209259





Epoch: 0 Generator Loss: 28.1228 Discriminator Loss: 0.5182862281799316




...




Epoch: 40 Generator Loss: 13.7681 Discriminator Loss: 0.00013107495033182204





Epoch: 40 Generator Loss: 13.4198 Discriminator Loss: 0.00050531787564978






Iter:3861 Generator Loss:15.3510 Discrimator Loss:0.0003 GA2B:0.9902 GB2A:0.9789 G_id:0.8343 G_cyc:0.9210 D_A:0.0005 D_B:0.0001: : 0it [00:08, ?it/s]




Epoch: 40 Generator Loss: 15.3510 Discriminator Loss: 0.00033984260517172515





Epoch: 40 Generator Loss: 13.8832 Discriminator Loss: 7.991894381120801e-05






Iter:3863 Generator Loss:13.9650 Discrimator Loss:0.0001 GA2B:0.9915 GB2A:0.9904 G_id:0.7565 G_cyc:0.8201 D_A:0.0000 D_B:0.0001: : 0it [00:09, ?it/s]




Epoch: 40 Generator Loss: 13.9650 Discriminator Loss: 9.404000593349338e-05


